
# 🧪 09 — RAG Projects Lab (Python + Node.js, Option A)

Welcome to the **RAG Projects Lab** — your project hub for the RAG Universe.

Use this notebook to:

- Choose **real RAG projects** to implement
- Follow **Python** and/or **Node.js** tracks
- Connect everything with:
  - core RAG pipeline (notebooks 04 & 05)
  - architectures (06)
  - evaluation (07)
  - domain patterns (08)
  - Agents & MCP universes (Agentic RAG, RAG + tools)

Each project is described with:

1. Problem statement  
2. User stories  
3. RAG pipeline design (ingest → index → retrieve → answer)  
4. Evaluation & logging considerations  
5. Python track blueprint  
6. Node.js track blueprint  
7. Milestones (v0, v1, v2)  
8. Extensions / advanced ideas  


## 📘 Project 1 — Simple Local Docs Q&A


### 1.1 Problem Statement

Build a **local-only RAG assistant** that can answer questions about a folder of documents:

- small PDFs, markdown, code, or policies
- no external APIs except the LLM itself (and optionally embeddings)

Use cases:

- “Explain our onboarding guide.”
- “Summarize the key risks in this internal policy.”



### 1.2 User Stories

- *As a new team member*, I want to ask questions about internal docs without reading them all.
- *As a student*, I want to ask questions about notes and exports from books/articles.



### 1.3 RAG Pipeline Design

Baseline pipeline:

1. **Ingestion**
   - load `.pdf`, `.md`, `.txt` from a directory
2. **Chunking**
   - `RecursiveCharacterTextSplitter` or similar
3. **Embedding**
   - OpenAI / local model
4. **Vector Store**
   - Chroma or FAISS (local)
5. **Retrieval**
   - kNN with `top_k` adjustable
6. **Generation**
   - LLM with citations (file paths, page numbers)
7. **Logging**
   - log: query, retrieved docs, answer

No agents, no MCP — just a clean, simple RAG.



### 1.4 Python Track — Blueprint

Use notebook **04_RAG_HandsOn_Python** as your base.

Suggested repo structure:

```text
projects/project_01_simple_rag_qna/
  python/
    app.py            # CLI or simple script for Q&A
    ingest.py         # load + chunk + index
    config.py
    rag_pipeline.py   # retrieval + answer
    data/             # your docs (git-ignored if private)
```

Implementation steps:

1. Copy the ingestion + chunking code from notebook 04.
2. Add a small CLI (e.g., `python app.py --query "..."`).
3. Save the vector store locally (persist directory).
4. Add a `--rebuild-index` flag to re-ingest when docs change.



### 1.5 Node.js Track — Blueprint

Use notebook **05_RAG_HandsOn_Node** as your base.

Suggested structure:

```text
projects/project_01_simple_rag_qna/
  node/
    src/
      config.js
      loaders.js
      chunker.js
      embeddings.js
      vectorStore.js
      rag.js
      index.js        # CLI entrypoint
```

Implementation steps:

1. Start from the `rag.js` and `index.js` examples.
2. Parameterize the doc folder via CLI args or env.
3. Log retrieved docs and answers.
4. Add a simple help message for usage.



### 1.6 Milestones

- **v0** — Q&A over a fixed folder of docs (no persistence, no CLI flags).  
- **v1** — Proper CLI, persisted index, configurable `top_k`.  
- **v2** — Add summary mode (`--mode summary`) to summarize an entire folder or single doc.  

### 1.7 Extensions

- Add support for **per-user folders**.
- Add a minimal web UI (e.g., FastAPI + simple HTML, or Node + Express).


## 🧪 Project 2 — Hybrid RAG (Dense + BM25)


### 2.1 Problem Statement

Improve retrieval quality by combining:

- **Dense** semantic search (embeddings)
- **Sparse** keyword/BM25 search

The goal is to handle:

- exact term matches
- synonyms and semantic variants



### 2.2 Pipeline Design

Hybrid retrieval:

1. Dense retriever:
   - vector store (Chroma/FAISS)
2. Sparse retriever:
   - BM25 / keyword index (e.g., `whoosh`, `elasticsearch`, or a simple in-memory index)
3. Fusion:
   - score or rank from both
   - simple heuristic: weighted sum, or interleaving

Then pass the fused top-k to the LLM.



### 2.3 Python Track — Blueprint

Extend:

- `rag_core/python/rag_pipeline/retriever.py`
- project folder: `project_02_hybrid_rag/python/`

Implementation ideas:

- Use `rank_bm25` or a simple BM25 implementation.
- Implement a `HybridRetriever` class that:
  - calls dense + sparse
  - merges results
- Add config for:
  - weights (dense vs sparse)
  - `top_k_dense`, `top_k_sparse`.



### 2.4 Node.js Track — Blueprint

Use Node-compatible tools:

- For sparse search:
  - simple inverted index
  - or an external service if desired (Elasticsearch / MeiliSearch)

Focus of this lab:

- designing the **fusion logic**
- keeping the interface clean:
  - `hybridRetrieve(query): Promise<docs[]>`



### 2.5 Milestones

- **v0** — Dense-only and BM25-only retrievers (two separate modes).  
- **v1** — Hybrid fusion with simple weighted sum.  
- **v2** — Add config & small evaluation comparing dense vs sparse vs hybrid.  


## 🎯 Project 3 — RAG with Reranker


### 3.1 Problem Statement

Improve the **precision** of final context by:

- retrieving a larger candidate set
- passing them through a **reranker** (cross-encoder or LLM-based)
- feeding only top filtered chunks to the generator



### 3.2 Pipeline Design

1. Retrieve top N (e.g., 30) via dense/hybrid retrieval.  
2. Rerank candidates by:
   - Cohere reranker
   - or LLM-as-reranker  
3. Keep top k (e.g., 5–10) for answer generation.  



### 3.3 Python Track — Blueprint

Create:

- `reranker.py` in `rag_core/python/rag_pipeline/`
  - `CohereReranker`
  - `LLMReranker` (calls an LLM to score chunks vs query)

Project folder: `project_04_rag_with_reranker/python/`.

Use eval notebook (07) to:

- compare recall/precision with and without reranking.



### 3.4 Node.js Track — Blueprint

Implement:

- `reranker.js` in `rag_core/node/src/pipeline/`
- optionally call:
  - Cohere rerank API
  - or use a JS LLM call as judge

Connect to a Node project folder:

- `project_04_rag_with_reranker/node/`



### 3.5 Milestones

- **v0** — Basic RAG w/o reranker (baseline).  
- **v1** — RAG with fixed reranker and logs of rerank scores.  
- **v2** — Add a small eval script to compare answer quality before/after.  


## 🤖 Project 4 — Agentic RAG (Multi-Step Retrieval)


### 4.1 Problem Statement

Upgrade from “one-shot retrieval” to **Agentic RAG**:

- An agent can:
  - reformulate queries
  - do multi-hop retrieval (“first find overview, then details”)
  - reason about whether more retrieval is needed



### 4.2 Roles & Patterns

Leverage your **Agents Universe**:

- RAG Worker Agent:
  - tool: `search_knowledge_base`
- Manager Agent:
  - decides:
    - “Should I query again?”
    - “Do I have enough context?”



### 4.3 Python Track — Blueprint

In `project_03_agentic_rag/python/`:

- Import or re-implement a simple Agent class (from Agents Universe).
- Wrap your RAG retriever as a **tool**.
- Implement a loop:
  - THINK → ACT (call tool) → OBSERVE → repeat until done → FINAL ANSWER.



### 4.4 Node.js Track — Blueprint

In `project_03_agentic_rag/node/`:

- Use a lightweight agent skeleton (from your Agents Node hands-on).
- Expose:
  - `ragSearchTool(query)`
  - `getChunkTool(id)` if needed.
- Implement a ReAct-style loop for 1–3 steps of retrieval.



### 4.5 Milestones

- **v0** — Single-step RAG with an Agent shell (no real reasoning).  
- **v1** — Multi-step Agentic RAG with 2–3 retrieval steps allowed.  
- **v2** — Add logging of thought/action/observation for debugging and evaluation.  


## 🏭 Project 5 — Domain RAG (Finance / Health / Legal / Code)


### 5.1 Problem Statement

Turn RAG from **generic** to **domain-specific**:

- Finance tutor
- Healthcare explainer (educational, not diagnostic)
- Legal clause navigator (informational only)
- Codebase assistant



### 5.2 Design Principles

From notebook **08_RAG_Real_World_Patterns**:

- Finetune chunking & retrieval by domain:
  - legal → clauses & section headings
  - finance → sections like risks, fees, examples
  - health → guidelines & glossary entries
  - code → functions, classes, files

- Add **safety wrappers**:
  - disclaimers
  - guardrails (e.g., refuse personal financial/medical/legal advice)



### 5.3 Python Track — Blueprint

For each domain, create a project:

- `project_05_domain_rag_finance/python/`
- `project_06_domain_rag_health/python/`
- `project_07_domain_rag_legal/python/`
- `project_08_codebase_rag_assistant/python/`

Focus on:

- domain-specific prompts
- domain-specific chunking/metadata (e.g., section titles)
- domain-specific eval samples



### 5.4 Node.js Track — Blueprint

Mirror the structure with Node projects:

- same domains
- same ingestion layout
- Node pipelines that call:
  - same vector stores
  - same patterns



### 5.5 Milestones

- **v0** — Domain-specific Q&A with disclaimers but generic indexing.  
- **v1** — Domain-aware chunking + metadata.  
- **v2** — Domain-specific evaluation set + metrics in notebook 07.  


## 📊 Project 6 — RAG Evaluation & Benchmark Suite


### 6.1 Problem Statement

Create a **reusable evaluation harness** to compare:

- different chunking strategies
- different embedding models
- different retrievers (dense vs hybrid)
- different prompts / LLMs



### 6.2 Components

- `data/eval/eval_questions.jsonl`
- `data/eval/gold_answers.jsonl` (optional)
- `scripts/eval_rag.py` or `.js` runner
- results written to:
  - `reports/eval/` as CSV/JSON



### 6.3 Python Track — Blueprint

Use ideas/code from `07_RAG_Evaluation_and_Benchmarks.ipynb`:

- `evaluate_rag_dataset()` style loop
- LLM-as-judge for faithfulness/helpfulness
- Optionally integrate with an external RAGAS-like library



### 6.4 Node.js Track — Blueprint

Implement:

- a Node CLI that:
  - reads questions
  - calls a RAG pipeline
  - optionally calls an LLM-judge
- saves metrics for comparison



### 6.5 Milestones

- **v0** — Evaluate one RAG config over small dataset.  
- **v1** — Evaluate multiple configs (e.g., chunk size 400 vs 800).  
- **v2** — Add simple dashboard or CSV plots to visualize metrics.  


## 🧬 Project 7 — Mini Full RAG + Agents + MCP


### 7.1 Problem Statement

Build a **small but complete** system that combines:

- RAG Universe
- Agents Universe
- MCP Universe

Flow:

- User → Manager Agent → RAG Worker (via MCP tools) → Answer + citations



### 7.2 Architecture

Use notebook **10_Full_RAG_Agents_MCP (Agents Universe)** as conceptual reference.

Concrete steps:

- Expose RAG as an MCP server (or mock tools).
- Have Manager Agent call RAG tools via MCP.
- Return a structured, cited answer.



### 7.3 Python & Node Tracks — Blueprint

You can choose:

- **Python + MCP** for a first end-to-end implementation
- or **Node + MCP** if your stack is heavily JS-based

Project suggestion:

```text
projects/mini_fullstack_rag_agents_mcp/
  python/...
  node/...
```

Focus on:

- keeping the contracts (tool signatures, prompts) identical across languages
- logging every step for debugging



## ✅ How to Use This Lab

1. Pick **one project** per week or month.  
2. For each project:
   - Decide: **Python**, **Node**, or **both**.  
   - Reuse code & patterns from:
     - 03 (concepts)
     - 04 / 05 (hands-on labs)
     - 06 (architectures)
     - 07 (evaluation)
     - 08 (domain patterns)
     - Agent & MCP notebooks (for cross-universe builds)  
3. Treat each finished project as:
   - a **portfolio artifact**
   - a **teaching example** for others
   - a **building block** in your “AI for Free” ecosystem.

This notebook turns your RAG Universe from **“world-class notes”** into a **pipeline of real, multi-language, production-style projects**.
